Given a HTML file with the results of a submission, load it into a DataFrame and 
perform basic plots.

One way to get the HTML file is to view the submission results, scroll down until the end of the list of games and save the page.


In [ ]:
!wget https://r-evolux.com/wagner.ikeda.html

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
def get_games(file):

    with open(file) as html:
        html_lines = html.readlines()
    submission = ''
    for i in range(len(html_lines)):
        if 'submission-' in html_lines[i]:
            submission_ix = html_lines[i].index('submission')
            temp = html_lines[i][submission_ix:]
            submission = temp.split('>')[0]
            submission = submission.split('#')[0]
            submission = submission.split('-')[-1]

    items = [item for item in html_lines if 'mdc-list-item' in item]
    items = items[1:]
    items = items[:-2]

    team = file.split('.')[0]
    games = []
    for i in range(len(items)):
        try:
            episode_ix = items[i].index('episode-')
            temp = items[i][episode_ix:]
            episode = temp.split('\"')[0].split('-')[-1]
            team1_ix = temp.index('[')
            team1_ = temp[team1_ix:].split('<')[0]
            team1 = team1_.split()[1]
            team1 = team1.lower()
            team1_score = team1_.split()[-2]
            temp = temp[team1_ix + 1:]
            team2_ix = temp.index('[')
            team2_ = temp[team2_ix:].split('<')[0]
            team2 = team2_.split()[1]
            team2 = team2.lower()
            team2_score = team1_.split()[-2]
            temp = temp[team2_ix:]
            when_ix = temp.index('ago')
            when = temp[:when_ix].split('>')[-1]
            if team == team1:
                score = team1_score
            else:
                score = team2_score
            games.append([submission, episode, team1.lower(), 
                          team1_score, team2.lower(), team2_score, score, when])
        except:
            print('não achou!')
    return team, games

In [ ]:
def get_df(games):
    df_games = pd.DataFrame(games)
    df_games.columns = ['submission', 'episode', 'team1', 'team1_score', 'team2', 
                        'team2_score', 'score', 'when']

    df_games = df_games.astype({'team1_score':int, 'team2_score':int, 'score':int})
    return df_games

In [ ]:
def get_scores(file):
    team, games = get_games(file)
    df = get_df(games)
    scores = [int(score) for score in df.score]
    scores.reverse()
    scores = np.array(scores)
    return team, scores

In [ ]:
def plot(file):
    _, scores = get_scores(file)
    plt.plot(np.arange(len(scores)), np.array(scores))

In [ ]:
plot('wagner.ikeda.html')

In [ ]:
file = 'wagner.ikeda.html'
team, games = get_games(file)
df = get_df(games)

In [ ]:
df